In [11]:
import utils 
from models.dnn import Net
import pandas as pd
import numpy as np 
from scipy.stats import pearsonr 
import  torch as th
import seaborn as sns 

In [12]:
import torch

In [13]:
tags = ["10:00:00","13:30:00","14:30:00"]

In [14]:
X_tests = [utils.df_test.query("timestamp==@tags[@i]").filter(regex='X_') for i in range(3)]
y_tests = [ utils.df_test.query("timestamp==@tags[@i]")["y_1"] for i in range(3)]

In [15]:
weights = [torch.load(f"/home/ray/workspace/proj_fcst/notebook/checkpoints/tmspc/2023-04-04_09-53/dnn_tmspc_{i}.pth") for i in tags ]
models = [Net(
        input_size=260,
        hidden_sizes=[200, 150, 100, 80, 20],
        output_size=1,
        lr=1e-4,
        act="SiLU",
    ) for _ in range(3)]
for i in range(3):
    models[i].load_state_dict(weights[i])

In [16]:
y_preds = [models[i](th.from_numpy(X_tests[i].values).float()).detach().numpy().flatten()/1e2 for i in range(3)]

In [17]:
timestamps = [utils.df_test.query("timestamp==@tags[@i]").date for i in range(3)]

In [18]:
y_preds[0]

array([-6.6146254e-06, -6.6034513e-04, -2.3997262e-04, ...,
        1.6449362e-03,  2.6676944e-03,  2.7873338e-04], dtype=float32)

In [19]:
y_tests[0]

4794944   -0.024857
4794945   -0.025547
4794946   -0.004008
4794947   -0.024125
4794948   -0.005238
             ...   
7601169   -0.032025
7601170    0.030727
7601171   -0.002963
7601172    0.003151
7601173    0.020352
Name: y_1, Length: 943004, dtype: float64

In [20]:
for i in range(3):
    print(f"{tags[i]}".center(20,"="))
    mmt = utils.ModelMetrics(y_preds[i],y_tests[i],timestamps[i])
    mmt.evaluate()

======10:00:00======


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type array(pyobject, 1d, C)
During: typing of argument at /home/ray/workspace/proj_fcst/notebook/utils.py (124)

File "utils.py", line 124:
def calculate_ic(y_hat, y, timestamps, unique_timestamps):
    ic_values = np.full_like(unique_timestamps, 0)
    ^


In [ ]:
df_y = [pd.DataFrame({'y_hat':y_preds[i],'y':y_tests[i]}) for i in range(3)]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(20,10))
for i in range(3):
    df_y[i].plot(kind='hist',bins=100,ax=axs[i],alpha=0.5)
    print(df_y[i].describe())